In [118]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier

In [119]:
df_train = pd.read_parquet("train_final.parquet")
df_test = pd.read_parquet("test_final.parquet")
df_sample_submission = pd.read_parquet("submission_sample_final.parquet")

In [120]:
df_train

,id,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,...,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49,target
0,5beefd4d2bf4a4767e0df8108,10,5245.571,981.182,205.948,VODAFONE TR,Apple,-1.197737,1.113360,-1.123334,...,-2.613336,-2.032903,2.645719,-1.023478,1.658986,-1.559406,-2.161336,30,58,"menu2, menu4, menu5"
1,867285b116c063d5a8482f5be,10,5184.876,557.650,487.587,TURKCELL,samsung,-2.336352,2.567766,-0.494908,...,-0.983938,-1.453756,-0.021547,-0.195770,2.775513,-0.318980,-4.291473,21,45,"menu7, menu8, menu4"
2,c82a7cbd2e00d9b66c06bcadc,10,3835.618,3275.128,43.806,TURK TELEKOM,Redmi,-2.561455,2.061736,-0.184511,...,-1.668703,-3.599403,1.673868,0.631790,1.293131,-2.230909,-2.383524,19,61,"menu2, menu8, menu4"
3,f2d2b25073ccc298eced86897,10,3532.544,154.509,64.724,TURKCELL,samsung,-2.529918,3.358050,-0.851366,...,-1.861418,-1.219658,1.863495,0.213170,1.029710,-1.142185,-4.466191,2,41,"menu6, menu2, menu1"
4,7818c92a58af0f2cb7c361738,10,3344.192,787.896,715.115,VODAFONE TR,samsung,-2.922361,2.096124,0.060796,...,-0.142903,-1.875545,1.024499,-0.186423,-0.061626,-1.462175,-2.371206,23,85,"menu6, menu2, menu8"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,2e54f32ced9fae6ef802ceaa2,12,44.397,43.425,41.678,O2 - DE,HUAWEI,-1.531534,2.596604,0.340233,...,-1.484154,-1.988186,0.044385,0.863442,0.761354,1.073711,-3.560019,29,68,"menu2, menu8, menu4"
94045,6aa4ff0f6cc5ef4c2980b2862,12,44.331,43.977,40.620,TURKCELL,samsung,-1.268987,2.300487,0.231711,...,-0.632773,-1.624605,2.406462,-0.340132,-0.893553,-2.061401,-1.908158,25,43,"menu9, menu2, menu5"
94046,fa842185a0edd210845b78308,12,44.142,43.591,41.736,VODAFONE TR,Apple,-1.950039,2.805681,0.438200,...,0.184886,-3.477337,0.449427,0.445361,-0.495204,-0.955097,-3.843092,4,25,"menu6, menu2, menu4"
94047,db8f55b8499f8d8c05148240e,12,43.963,43.350,40.862,TURK TELEKOM,Redmi,-2.389140,2.358281,0.683524,...,-0.186062,-2.329308,0.864857,1.284815,-0.100755,-1.056479,-1.452037,25,60,"menu6, menu8, menu4"


In [121]:
df_test

,id,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,...,feature_40,feature_41,feature_42,feature_43,feature_44,feature_45,feature_46,feature_47,feature_48,feature_49
0,2e6105f5911256f4f6c4813ed,1,6893.544,246.854,242.636,VODAFONE TR,samsung,-1.723524,3.216489,-1.138474,...,-1.094519,-1.217407,-4.280456,1.512240,-2.306445,2.066388,0.844927,-1.026193,18,58
1,c56ad71dae0a5dbd3e7d36adc,1,4481.065,740.209,263.860,TURKCELL,Apple,-0.417275,2.024433,0.102952,...,1.806486,-3.477517,-2.064966,1.499805,1.284697,0.189269,-1.563224,-1.901654,3,35
2,4d02ea175f6581f0c6385311f,1,4340.702,2742.163,318.700,TURKCELL,samsung,-2.943294,2.769536,0.734942,...,1.759080,-2.038839,-2.067219,2.141083,0.055355,0.084739,-1.009925,-2.058473,7,50
3,3412d27a86c286ba078fa935c,1,4129.666,181.397,155.423,TURK TELEKOM,Apple,-2.346902,2.684752,0.168206,...,2.171847,-0.925040,-1.484278,0.666036,0.911519,0.616167,0.092304,-1.874706,22,47
4,0203b561f6f7e10eafa46eefa,1,3903.944,126.133,100.060,TURKCELL,POCO,-1.745354,2.355863,0.318961,...,-0.373413,-0.015773,-2.961445,1.301413,1.375090,-0.107355,0.924390,-1.606419,29,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11950,7687113f46112edf4f56666ee,1,47.726,40.879,40.647,TURKCELL,HUAWEI,-2.952961,1.352361,-0.396846,...,0.333993,-0.533786,-1.779781,0.228962,0.403641,-0.906588,-0.788647,-0.627803,18,45
11951,5ff8eb7a06fd48b60dbc04f34,1,47.471,44.745,41.040,TURKCELL,Apple,-1.945921,2.003634,-0.300293,...,1.468012,-1.345186,-1.067598,1.685431,0.879146,0.206344,-1.623952,-2.496700,20,44
11952,ac23a7b9ad3e5d61e738c854b,1,47.403,45.832,43.932,TURK TELEKOM,Redmi,-1.596234,3.165674,-1.520392,...,1.096878,-0.558995,-3.029989,1.352324,0.351689,-0.326579,-0.382071,-2.995726,17,34
11953,7da05018634ea2eee4b122756,1,46.044,43.592,40.735,TURK TELEKOM,samsung,-1.984582,2.382992,-1.060896,...,1.006593,-2.548628,-2.803372,1.091970,0.436466,0.235511,-1.625251,-2.354214,26,58


In [122]:
df_sample_submission

,id,target
0,2e6105f5911256f4f6c4813ed,
1,c56ad71dae0a5dbd3e7d36adc,
2,4d02ea175f6581f0c6385311f,
3,3412d27a86c286ba078fa935c,
4,0203b561f6f7e10eafa46eefa,
...,...,...
11950,7687113f46112edf4f56666ee,
11951,5ff8eb7a06fd48b60dbc04f34,
11952,ac23a7b9ad3e5d61e738c854b,
11953,7da05018634ea2eee4b122756,


In [123]:
def detectOs(x):
    if x == "Apple":
        return "iOS"
    else:
        return "Android"
    
df_train["Os"] = df_train["devicebrand"].apply(lambda x:detectOs(x))
df_test["Os"] = df_test["devicebrand"].apply(lambda x:detectOs(x))

In [124]:
def targetSplit(x,i):
    x = x.split(", ")
    return x[i]
    
df_train["First_Menu"] = df_train["target"].apply(lambda x:targetSplit(x,0))
df_train["Second_Menu"] = df_train["target"].apply(lambda x:targetSplit(x,1))
df_train["Third_Menu"] = df_train["target"].apply(lambda x:targetSplit(x,2))
df_train.drop(columns=['target'],inplace=True)

In [125]:
df_train.loc[(df_train["carrier"]!="TURKCELL")&
             (df_train["carrier"]!="VODAFONE TR")&
             (df_train["carrier"]!="TURK TELEKOM"),"carrier"] = "Other"

df_test.loc[(df_test["carrier"]!="TURKCELL")&
             (df_test["carrier"]!="VODAFONE TR")&
             (df_test["carrier"]!="TURK TELEKOM"),"carrier"] = "Other"

In [126]:
df_train.loc[(df_train["devicebrand"]!="Apple")&
             (df_train["devicebrand"]!="samsung")&
             (df_train["devicebrand"]!="Redmi")&
             (df_train["devicebrand"]!="HUAWEI")&
             (df_train["devicebrand"]!="xiaomi")&
             (df_train["devicebrand"]!="OPPO"),"devicebrand"] = "Other"

df_test.loc[(df_test["devicebrand"]!="Apple")&
             (df_test["devicebrand"]!="samsung")&
             (df_test["devicebrand"]!="Redmi")&
             (df_test["devicebrand"]!="HUAWEI")&
             (df_test["devicebrand"]!="xiaomi")&
             (df_test["devicebrand"]!="OPPO"),"devicebrand"] = "Other"

In [127]:
df_train["n_seconds_1_2_sum"] = df_train[["n_seconds_1","n_seconds_2"]].sum(axis=1)
df_train["n_seconds_1_3_sum"] = df_train[["n_seconds_1","n_seconds_3"]].sum(axis=1)
df_train["n_seconds_2_3_sum"] = df_train[["n_seconds_2","n_seconds_3"]].sum(axis=1)
df_train["n_seconds_sum"] = df_train.sum(axis=1)
df_train["n_seconds_mean"] = df_train.sum(axis=1)/3

df_test["n_seconds_1_2_sum"] = df_test[["n_seconds_1","n_seconds_2"]].sum(axis=1)
df_test["n_seconds_1_3_sum"] = df_test[["n_seconds_1","n_seconds_3"]].sum(axis=1)
df_test["n_seconds_2_3_sum"] = df_test[["n_seconds_2","n_seconds_3"]].sum(axis=1)
df_test["n_seconds_sum"] = df_test.sum(axis=1)
df_test["n_seconds_mean"] = df_test.sum(axis=1)/3

C:\Users\Orhun\AppData\Local\Temp\ipykernel_2528\1312673939.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_train["n_seconds_sum"] = df_train.sum(axis=1)
C:\Users\Orhun\AppData\Local\Temp\ipykernel_2528\1312673939.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_train["n_seconds_mean"] = df_train.sum(axis=1)/3
C:\Users\Orhun\AppData\Local\Temp\ipykernel_2528\1312673939.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_test["n_seconds_sum"] = df_test.sum(axis=1)
C:\Us

In [128]:
def label_encoding(df,column_name):
    label_encoder = LabelEncoder()
    df[column_name] = label_encoder.fit_transform(df[column_name])

In [129]:
def one_hotcoding(df,column_name):
    one_hotcoder = pd.get_dummies(df[column_name])
    return one_hotcoder

In [130]:
label_encoding(df_train,"carrier")
label_encoding(df_train,"devicebrand")
label_encoding(df_train,"Os")
label_encoding(df_train,"First_Menu")
label_encoding(df_train,"Second_Menu")
label_encoding(df_train,"Third_Menu")

label_encoding(df_test,"carrier")
label_encoding(df_test,"devicebrand")
label_encoding(df_test,"Os")

In [131]:
df_train.drop(columns=["id"],inplace=True)


x = df_train.drop(columns=["First_Menu","Second_Menu","Third_Menu"])
y = df_train[["First_Menu","Second_Menu","Third_Menu"]]

X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [132]:
x

,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,feature_3,...,feature_46,feature_47,feature_48,feature_49,Os,n_seconds_1_2_sum,n_seconds_1_3_sum,n_seconds_2_3_sum,n_seconds_sum,n_seconds_mean
0,10,5245.571,981.182,205.948,3,0,-1.197737,1.113360,-1.123334,-0.263580,...,-1.559406,-2.161336,30,58,1,6226.753,5451.519,1187.130,19391.884085,12927.922723
1,10,5184.876,557.650,487.587,2,5,-2.336352,2.567766,-0.494908,0.949101,...,-0.318980,-4.291473,21,45,0,5742.526,5672.463,1045.237,18761.558217,12507.705478
2,10,3835.618,3275.128,43.806,1,4,-2.561455,2.061736,-0.184511,1.062306,...,-2.230909,-2.383524,19,61,0,7110.746,3879.424,3318.934,21549.151333,14366.100889
3,10,3532.544,154.509,64.724,2,5,-2.529918,3.358050,-0.851366,1.643876,...,-1.142185,-4.466191,2,41,0,3687.053,3597.268,219.233,11303.693379,7535.795586
4,10,3344.192,787.896,715.115,3,5,-2.922361,2.096124,0.060796,-1.487557,...,-1.462175,-2.371206,23,85,0,4132.088,4059.307,1503.011,14655.294259,9770.196173
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94044,12,44.397,43.425,41.678,0,1,-1.531534,2.596604,0.340233,-1.149720,...,1.073711,-3.560019,29,68,0,87.822,86.075,85.103,492.248746,328.165831
94045,12,44.331,43.977,40.620,2,5,-1.268987,2.300487,0.231711,0.741582,...,-2.061401,-1.908158,25,43,0,88.308,84.951,84.597,463.663261,309.108841
94046,12,44.142,43.591,41.736,3,0,-1.950039,2.805681,0.438200,2.976427,...,-0.955097,-3.843092,4,25,1,87.733,85.878,85.327,424.449145,282.966097
94047,12,43.963,43.350,40.862,1,4,-2.389140,2.358281,0.683524,0.234449,...,-1.056479,-1.452037,25,60,0,87.313,84.825,84.212,477.087458,318.058305


In [133]:
y

,First_Menu,Second_Menu,Third_Menu
0,1,3,3
1,6,6,2
2,1,6,2
3,5,1,0
4,5,1,6
...,...,...,...
94044,1,6,2
94045,8,1,3
94046,5,1,2
94047,5,6,2


In [134]:
X_train

,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,feature_3,...,feature_46,feature_47,feature_48,feature_49,Os,n_seconds_1_2_sum,n_seconds_1_3_sum,n_seconds_2_3_sum,n_seconds_sum,n_seconds_mean
25787,11,467.913,310.153,277.905,3,0,-1.681805,2.142508,0.562797,-1.161826,...,-1.503813,-2.185377,31,48,1,778.066,745.818,588.058,3253.482083,2168.988055
72740,12,310.811,280.096,273.160,2,0,-1.790492,2.127541,1.071017,-1.208557,...,-1.056161,-3.069718,24,62,1,590.907,583.971,553.256,2685.886283,1790.590855
39526,11,201.131,145.928,47.916,3,5,-2.761982,0.012119,-0.135885,-1.355689,...,-0.054689,-2.110920,28,52,0,347.059,249.047,193.844,1271.176182,847.450788
45165,11,132.749,118.371,114.088,1,5,-1.738869,1.273165,0.061282,0.667869,...,-0.788386,-2.958128,19,41,0,251.120,246.837,232.459,1161.701902,774.467935
19376,11,758.930,288.933,186.044,2,0,-2.223320,1.023573,1.663910,0.056099,...,-1.599712,-3.744726,5,29,1,1047.863,944.974,474.977,3742.279990,2494.853326
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6265,10,307.836,251.927,118.365,0,5,1.238334,1.737300,-0.084119,-0.398327,...,-0.560315,-2.630362,14,72,0,559.763,426.201,370.292,2126.002038,1417.334692
54886,12,851.542,130.979,85.147,2,3,-1.532209,1.846237,1.084361,-1.060927,...,-0.430353,-2.578184,22,37,0,982.521,936.689,216.126,3269.234608,2179.489739
76820,12,258.191,104.041,89.864,1,0,-1.862614,1.710194,0.050358,-1.470732,...,-1.105424,-1.774638,36,57,1,362.232,348.055,193.905,1456.512763,971.008509
860,10,857.067,287.388,279.076,1,5,-2.429906,1.634941,0.988270,-1.836210,...,1.095616,-2.203076,31,69,0,1144.455,1136.143,566.464,4375.330046,2916.886698


In [135]:
Y_train

,First_Menu,Second_Menu,Third_Menu
25787,5,3,0
72740,5,1,6
39526,8,1,2
45165,5,1,3
19376,5,1,3
...,...,...,...
6265,5,1,0
54886,5,1,2
76820,5,1,6
860,8,6,2


In [136]:
X_test

,month,n_seconds_1,n_seconds_2,n_seconds_3,carrier,devicebrand,feature_0,feature_1,feature_2,feature_3,...,feature_46,feature_47,feature_48,feature_49,Os,n_seconds_1_2_sum,n_seconds_1_3_sum,n_seconds_2_3_sum,n_seconds_sum,n_seconds_mean
71983,12,322.099,190.918,127.217,2,5,-2.223116,0.946927,1.107268,-2.571088,...,0.393778,-2.455410,26,64,0,513.017,449.316,318.135,2017.931972,1345.287981
24607,11,507.390,65.318,48.413,1,5,-1.291186,1.739034,0.503340,-0.689582,...,-0.578993,-2.639758,33,53,0,572.708,555.803,113.731,1954.657361,1303.104907
55047,12,838.672,432.181,121.402,3,4,-1.927184,1.657606,0.691318,-0.072013,...,-0.741580,-2.055614,34,59,0,1270.853,960.074,553.583,4277.543391,2851.695594
17346,11,1020.229,258.357,187.167,0,3,-1.793335,0.632174,-0.721243,-0.590299,...,-1.809589,-1.877998,26,53,0,1278.586,1207.396,445.524,4482.040937,2988.027291
68865,12,375.914,366.845,155.577,2,0,-1.202234,1.514095,0.476541,-0.138625,...,-1.269625,-3.029242,16,59,1,742.759,531.491,522.422,2776.747836,1851.165224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80465,12,216.126,182.617,169.682,2,5,-1.230673,2.428997,0.825009,-0.968376,...,0.699808,-1.308315,11,77,0,398.743,385.808,352.299,1800.989560,1200.659707
59726,12,596.786,137.695,65.673,3,5,-2.626715,1.322484,-0.889738,-0.178716,...,-1.165260,-2.318295,15,37,0,734.481,662.459,203.368,2460.005206,1640.003470
81005,12,210.314,79.150,53.961,2,0,-2.426843,2.036848,-0.181062,-1.126848,...,-0.947266,-1.695404,24,65,1,289.464,264.275,133.111,1127.000884,751.333923
10243,10,185.012,67.814,44.382,2,0,-1.889391,2.445890,-0.387656,-0.373507,...,-1.319142,-2.202921,13,63,1,252.826,229.394,112.196,973.499009,648.999339


In [137]:
Y_test

,First_Menu,Second_Menu,Third_Menu
71983,5,7,0
24607,5,1,0
55047,5,1,3
17346,5,3,7
68865,8,4,0
...,...,...,...
80465,5,2,1
59726,1,3,3
81005,1,0,3
10243,1,3,0


In [138]:
model = MultiOutputClassifier(RandomForestClassifier(n_estimators=100, random_state=42))
model.fit(X_train, Y_train)
prediction = model.predict(X_test)

In [139]:
values = Y_test.to_numpy()

In [140]:
values

array([[5, 7, 0],
       [5, 1, 0],
       [5, 1, 3],
       ...,
       [1, 0, 3],
       [1, 3, 0],
       [5, 1, 3]])

In [141]:
prediction

array([[5, 1, 2],
       [5, 1, 6],
       [5, 1, 6],
       ...,
       [5, 1, 2],
       [5, 1, 2],
       [5, 1, 2]])

In [142]:
def numpy_to_binary(numpy_array):
    binary_array = []

    for array in numpy_array:
        binary = [0,0,0,0,0,0,0,0,0]
        for j in range(len(array)):
            binary[array[j]] = 1
        binary_array.append(binary)
    
    return np.array(binary_array)

In [143]:
def jaccard_similarity_score(values,prediction):
    jaccard_score_list = []
    for i in range(len(values)):
        intersection = np.sum(np.logical_and(values[i],prediction[i]))
        union = np.sum(np.logical_or(values[i],prediction[i]))

        jaccard_score = intersection / union

        jaccard_score_list.append(jaccard_score)

    return sum(jaccard_score_list)/len(jaccard_score_list)

In [144]:
values_bin = numpy_to_binary(values)
prediction_bin = numpy_to_binary(prediction)

jaccard_similarity_score(values_bin, prediction_bin)

0.528072833599154

In [145]:
df_test = df_test.drop(columns=["id"])

In [146]:
prediction = model.predict(df_test)

In [147]:
prediction

array([[5, 1, 2],
       [5, 1, 2],
       [5, 1, 3],
       ...,
       [5, 1, 2],
       [5, 1, 2],
       [5, 1, 3]])

In [148]:
result = numpy_to_binary(prediction)

In [149]:
result = result.tolist()

In [150]:
submission = pd.DataFrame({"id": df_sample_submission["id"],"target": result})

In [151]:
submission

,id,target
0,2e6105f5911256f4f6c4813ed,"[0, 1, 1, 0, 0, 1, 0, 0, 0]"
1,c56ad71dae0a5dbd3e7d36adc,"[0, 1, 1, 0, 0, 1, 0, 0, 0]"
2,4d02ea175f6581f0c6385311f,"[0, 1, 0, 1, 0, 1, 0, 0, 0]"
3,3412d27a86c286ba078fa935c,"[0, 1, 1, 0, 0, 1, 0, 0, 0]"
4,0203b561f6f7e10eafa46eefa,"[0, 1, 0, 0, 0, 1, 1, 0, 0]"
...,...,...
11950,7687113f46112edf4f56666ee,"[0, 1, 1, 0, 0, 0, 0, 0, 1]"
11951,5ff8eb7a06fd48b60dbc04f34,"[0, 1, 1, 0, 0, 1, 0, 0, 0]"
11952,ac23a7b9ad3e5d61e738c854b,"[0, 1, 1, 0, 0, 1, 0, 0, 0]"
11953,7da05018634ea2eee4b122756,"[0, 1, 1, 0, 0, 1, 0, 0, 0]"


In [152]:
def fix(x):
    x = str(x).replace("[","")
    x = str(x).replace(",","")
    x = str(x).replace("]","")
    x = str(x).replace(" ","")


    return x

submission["target"] = submission["target"].apply(lambda x:fix(x))

In [153]:
submission

,id,target
0,2e6105f5911256f4f6c4813ed,011001000
1,c56ad71dae0a5dbd3e7d36adc,011001000
2,4d02ea175f6581f0c6385311f,010101000
3,3412d27a86c286ba078fa935c,011001000
4,0203b561f6f7e10eafa46eefa,010001100
...,...,...
11950,7687113f46112edf4f56666ee,011000001
11951,5ff8eb7a06fd48b60dbc04f34,011001000
11952,ac23a7b9ad3e5d61e738c854b,011001000
11953,7da05018634ea2eee4b122756,011001000


In [154]:
submission.to_csv('submission.csv', index=False)

In [155]:
def count_non_three_ones(df, column_name='target'):
    count = 0
    for target in df[column_name]:
        if target.count('1') != 3:
            count += 1
    return count

count_non_three_ones(submission)

31